# Data Parallelism

- 이번 세션에는 데이터 병렬화 기법에 대해 알아보겠습니다.

## 1. `torch.nn.DataParallel`
- Data Parallelism은 `all-reduce` 연산을 활용하기 전과 후로 나뉩니다.
- 가장 먼저 우리에게 친숙한 `torch.nn.DataParallel`의 동작 방식에 대해 알아봅시다.
- `torch.nn.DataParallel`은 single-node & multi-GPU에서 동작하는 multi-thread 모듈입니다.

### 1) Forward Pass

1. 입력된 mini-batch를 **Scatter**하여 각 디바이스로 전송.
2. GPU-1에 올라와 있는 모델의 파라미터를 GPU-2,3,4로 **Broadcast**.
3. 각 디바이스로 복제된 모델로 Forward하여 출력값을 구함.
4. 구해진 출력값들을 **Gather**하여 GPU-1에 모음.

![](../images/dp_forward.png)

<br>

- 코드로 나타내면 아래와 같습니다.

In [3]:
def data_parallel(module, input, device_ids, output_device):
    inputs = nn.parallel.scatter(input, device_ids)
    # 입력 데이터를 device_ids들에 Scatter함

    replicas = nn.parallel.replicate(module, device_ids)
    # 모델을 device_ids들에 복제함.
   
    outputs = nn.parallel.parallel_apply(replicas, inputs)
    # 각 device에 복제된 모델이 각 device의 데이터를 Forward함.

    return nn.parallel.gather(outputs, output_device)
    # 모델의 출력값을 output_device(하나의 device)로 모음

### 2) Backward Pass

1. GPU-1에 모여있는 출력값과 라벨을 이용하여 GPU-1에서 모든 Loss를 계산.
2. 계산된 각각의 Loss를 각 디바이스에 **Scatter**함.
3. 전달받은 Loss를 이용해서 각 디바이스에서 Backward를 수행.
4. 계산된 모든 Gradient를 GPU-1로 **Reduce**하여 GPU-1의 모델을 업데이트.

![](../images/dp_backward.png)

<br>

### 혹시나 모르시는 분들을 위해...
- `loss.backward()` 기울기를 미분해서 Gradient를 계산
- `optimizer.step()` 계산된 Gradient를 이용해서 파라미터를 업데이트
- Computation cost는 `backward()` > `step()`.

![](../images/dp_backward_loss.png)


In [ ]:
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer
from datasets import load_dataset

# 1. create dataset
datasets = load_dataset("multi_nli").data["train"]
datasets = [
    {
        "premise": str(p),
        "hypothesis": str(h),
        "label": l.as_py(),
    }
    for p, h, l in zip(datasets[2], datasets[5], datasets[9])
]
data_loader = DataLoader(datasets, batch_size=32, num_workers=4)

# 2. create model and tokenizer
model_name = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3).cuda()

# 3. make data parallel module
# device_ids: 사용할 디바이스 리스트 / output_device: 출력값을 모을 디바이스
model = nn.DataParallel(model, device_ids=[0, 1, 2, 3], output_device=0)

# 4. create optimizer
optimizer = Adam(model.parameters(), lr=3e-5)

# 5. start training
for i, data in enumerate(data_loader):
    optimizer.zero_grad()
    tokens = tokenizer(
        data["premise"],
        data["hypothesis"],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    loss = model(
        input_ids=tokens.input_ids.cuda(),
        attention_mask=tokens.attention_mask.cuda(),
        labels=data["label"].cuda(),
    ).loss.sum()

    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(f"step:{i}, loss:{loss}")

    if i == 300:
        break

In [5]:
!python ../src/data_parallel.py

Using custom data configuration default
Reusing dataset multi_nli (/home/ubuntu/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39)
100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 67.81it/s]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceC

![](../images/dp_training.png)

- 0번 GPU에 Output들이 모두 Gather 되기 때문에 사용량이 많은 것을 알 수 있음.
- 만약 여러대의 GPU를 사용할 때, 특정 GPU의 용량이 많다면 `output_device`를 변경하여 출력을 원하는 곳으로 모을 수 있음.
  - 예를 들어 0번 GPU가 8GB, 1번 GPU가 24GB라면 용량이 큰 1번 GPU에 모으는 것이 효율적.
  - 이런 경우에 `output_device=1`와 같이 설정하여 1번 GPU로 출력을 모두 모을 수 있음.

<br>

## 2. `torch.nn.DataParallel`의 문제점


### 1) 멀티쓰레드 모듈이기 때문에 Python에서 비 효율적.
- Python은 GIL (Global Interpreter Lock)에 의해 하나의 프로세스에서 동시에 여러개의 쓰레드가 작동 할 수 없음.
- 따라서 근본적으로 멀티 쓰레드가 아닌 **멀티 프로세스 프로그램**으로 만들어서 여러개의 프로세스를 동시에 실행하게 해야함.

<br>

### 2) 메모리 불균형이 일어나서 GPU를 100% 활용할 수 없음.
- Output이 하나의 디바이스로 모이고, 한 디바이스가 모든 Loss를 계산하기 때문.
- **Output을 하나의 디바이스로 모으지 않고 각 디바이스에서 자체적으로 Loss를 계산**하게 만들어야 함.
- 따라서 아래와 같이 연산을 수정해야 함.

### Parallel Loss Forward

1. 입력된 mini-batch를 **Scatter**하여 각 디바이스로 전송.
2. GPU-1에 올라와 있는 모델의 파라미터를 GPU-2,3,4로 **Broadcast**.
3. 각 디바이스로 복제된 모델로 Forward하여 출력값을 구함.
4. <s>구해진 출력값들을 **Gather**하여 GPU-1에 모음.</s> → **각 디바이스에서 자체적으로 Loss를 계산함.**

![](../images/parallel_dp_forward.png)
위: 기존 Data Parallel / 아래: 변경된 Data Parallel

<br>

### Parallel Loss Backward

1. <s>GPU-1에 모여있는 출력값과 라벨을 이용하여 GPU-1에서 모든 Loss를 계산.</s> → **각 디바이스에서 자체적으로 Gradient를 계산함.**
2. <s>계산된 각각의 Loss를 각 디바이스에 **Scatter**함.</s> → 생략
3. 각 디바이스에서 Backward를 수행.
4. 계산된 모든 Gradient를 GPU-1로 **Reduce**하여 GPU-1의 모델을 업데이트.

![](../images/parallel_dp_backward.png)

위: 기존 Data Parallel / 아래: 변경된 Data Parallel

<br>

### 3) 하나의 모델에서 업데이트 된 모델이 다른 device로 매 스텝마다 복제되어야 함.
- Gradient를 Reduce하지 않고 평균을 계산해서 모든 device로 전송할 수 있다면 해결될 문제 → All-Reduce
- Gradient의 평균

<br>
